# Random Forests
In this workbook, we will examine **random forests**.   As you might expect, random forests are a collection of **decision trees**.   More precisely, random forests are an ensemble of decision trees, designed to overcome two drawbacks of decsion trees:
1.  Decision trees are prone to overfitting the data.  Now this is a limitation that can be overcome, as we saw in the previous example, but may be more difficult depending on sample size and the nature of the problem you are working on.
2.  Decision trees are sensitive to the actual distribution of the training data, such that small perturbations in one single distribution can yield a very different tree.

The basic idea is very simple: instead of training a single tree, we will train a large number of trees and average the results.   But of course if we use the same data, we will get the same trees.   To overcome this, **random** forests introduce the following ideas:
1.  Randomize the training data.   The technique used is called **bootstrap aggregating** or **bagging** for short.   If we have **n** samples in our training data, we select **n** samples **with replacement** from that same dataset.   On average this will choose about 63% of the training data for each tree (meaning that there are copies in the training set).   The key idea is that each tree in the forest sees a different (but overlapping) subset of the data.
2.  Randomize the features used by each tree in the forest.  By default, the random forests in scikit use only a subset of the available features (by default the rounded sqrt(number of input features)) when deciding which feature to use to split the data at each node in a given tree.

How are the results of all of these tree combined?   As we saw with our decision tree workbook, a decision tree can predict a class (0,1,2, etc), or a set of probabilities.   When combining an ensemble of trees we have two options to predict the result for a single sample:
1.  Hard voting: use the class predicted by most of the trees.
2.  Soft voting: average the probabilites, and use the class with the highest probability.   This allows higher performance by giving greater weight to more confident predicitons.   Sklearn uses this method for random forests.

# Feature Importance
A very useful property of random forests is that they can be used to assess **feature importance**.   The idea is that since different trees use different subsets of available features at each split point within each tree, we have a mechanism for assessing how important a specific feature is for our model.   A feature's importance is related to how much that feature reduces impurity on average across all of the trees in the forest.

To help us test this idea, we will add a **random** feature to our dataset.   This feature should neither help nor hurt our classification, and when we  check our feature importances later, we would expect that the random feature will be ranked very low in feature importance.


# Getting the data:
We will again use the puslar data as we did for the decision tree workbook.   The data come from the UCI Machine Learning Repository: https://archive.ics.uci.edu/ml/datasets/HTRU2

As usual the data is on github: 'https://raw.githubusercontent.com/big-data-analytics-physics/data/master/HTRU2/HTRU_2a.csv'

As noted above, will will add a separate column labeled "random" which is essentailly a *fake* feature. 

In [14]:
import pandas as pd
import numpy as np


#
# Read in all of the other digits
fname = 'https://raw.githubusercontent.com/big-data-analytics-physics/data/master/HTRU2/HTRU_2a.csv'
dfAll = pd.read_csv(fname)
#
# Add a feature called "random" with random numbers from 0-1
dfAll['random'] = np.random.random(size=len(dfAll))
dfAll.head(5)




,Profile_mean,Profile_stdev,Profile_skewness,Profile_kurtosis,DM_mean,DM_stdev,DM_skewness,DM_kurtosis,class,random
0,140.562500,55.683782,-0.234571,-0.699648,3.199833,19.110426,7.975532,74.242225,0,0.720577
1,102.507812,58.882430,0.465318,-0.515088,1.677258,14.860146,10.576487,127.393580,0,0.828634
2,103.015625,39.341649,0.323328,1.051164,3.121237,21.744669,7.735822,63.171909,0,0.947930
3,136.750000,57.178449,-0.068415,-0.636238,3.642977,20.959280,6.896499,53.593661,0,0.180099
4,88.726562,40.672225,0.600866,1.123492,1.178930,11.468720,14.269573,252.567306,0,0.132220


## Reorder data columns
Our code below expects the **last** column to have the **class** label.  Since we added the **random** column this is not longer true, so in the code below we reorder the columns.

In [15]:
dfAll = dfAll[['Profile_mean','Profile_stdev','Profile_skewness','Profile_kurtosis','DM_mean',
        'DM_stdev','DM_skewness','DM_kurtosis','random','class']]
num_features = 9
dfAll.head(5)

,Profile_mean,Profile_stdev,Profile_skewness,Profile_kurtosis,DM_mean,DM_stdev,DM_skewness,DM_kurtosis,random,class
0,140.562500,55.683782,-0.234571,-0.699648,3.199833,19.110426,7.975532,74.242225,0.720577,0
1,102.507812,58.882430,0.465318,-0.515088,1.677258,14.860146,10.576487,127.393580,0.828634,0
2,103.015625,39.341649,0.323328,1.051164,3.121237,21.744669,7.735822,63.171909,0.947930,0
3,136.750000,57.178449,-0.068415,-0.636238,3.642977,20.959280,6.896499,53.593661,0.180099,0
4,88.726562,40.672225,0.600866,1.123492,1.178930,11.468720,14.269573,252.567306,0.132220,0


# Defining Signal
The **class** variable distinguished signal from background.   As usual,  **1** is signal (pulsars) and **0** is background.   We make sure the sample is balanced so we have equal numbers of pulsars and non-pulsars.

In [3]:
#
# The data already has a 0/1 class variable that defines signal (1) and background (0)
#
# The data is already combined but it will be usefull to split it so we can look at 
# signal and background separately.
dfA = dfAll[dfAll['class']==1]
dfB = dfAll[dfAll['class']==0]

print("Length of signal sample:     ",len(dfA))
print("Length of background sample: ",len(dfB))

#
# Shuffle the data here
from sklearn.utils import shuffle
dfBShuffle = shuffle(dfB)
#
# Uncomment the next line to limit dfB to be the same length as dfA
#dfB_use = dfBShuffle
dfB_use = dfBShuffle.head(len(dfA))


dfCombined = dfB_use
dfCombined = pd.concat([dfCombined, dfA])
dfCombined = shuffle(dfCombined)

print("Size of signal sample ",len(dfA))
print("Size of background sample ",len(dfB_use))
print("Size of combined sample ",len(dfCombined))



Length of signal sample:      1639
Length of background sample:  16259
Size of signal sample  1639
Size of background sample  1639
Size of combined sample  3278


# Some useful methods
We have our usual helpful methods of nested_defaultdict and matplotlib, as well as getDecisionTreeGraphic, which we can use to visualize an individual tree within a forest.

In [4]:
from collections import defaultdict
from functools import partial
from itertools import repeat
def nested_defaultdict(default_factory, depth=1):
    result = partial(defaultdict, default_factory)
    for _ in repeat(None, depth - 1):
        result = partial(defaultdict, result)
    return result()

from plotly.offline import init_notebook_mode
init_notebook_mode(connected = True)
%matplotlib inline


def getDecisionTreeGraphic(estimator,feature_names,class_names):
  from sklearn import tree
  from io import StringIO
  import pydot_ng as pydot 
  import graphviz
  import matplotlib.pyplot as plt

  dot_data = StringIO()

  #                         class_names=classColumn,
  out = tree.export_graphviz(estimator,out_file=None,
                           feature_names=feature_names,
                           class_names=class_names,
                           filled=True, rounded=True,
                           special_characters=True,
                            node_ids=1,)
  import pydotplus
  pydot_graph = pydotplus.graph_from_dot_data(out)
  pydot_graph.set_size('"7,7!"')
  #print(pydot_graph.getvalue())
  #graph = graphviz.Source( out)
  graph = graphviz.Source(pydot_graph.to_string())
  return graph


# Performance Method

In [5]:
#
# Determine the performance
def binaryPerformance(y,y_pred,y_score,debug=False):
#
# Assuming a binary classifier with 1=signal, 0=background
  confusionMatrix = nested_defaultdict(int,2)
  for i in range(len(y_pred)):
    trueClass = y[i]
    predClass = y_pred[i]
    confusionMatrix[trueClass][predClass] += 1

  if debug:
    for trueClass in range(2):
      print("True: ",trueClass,end="")
      for predClass in range(2):
        print("\t",confusionMatrix[trueClass][predClass],end="")
      print()
    print()
  TP = confusionMatrix[1][1]
  FP = confusionMatrix[0][1]
  FN = confusionMatrix[1][0]
  TN = confusionMatrix[0][0]
  
  if debug:
    print("TP predicted true, actually true   ",TP)
    print("FP predicted true, acutally false  ",FP)
    print("TN predicted false, actually false ",TN)
    print("FN predicted false, actually true  ",FN)


  precision = TP / (TP + FP)
  recall = TP / (TP + FN)
  f1_score = 2.0 / ( (1.0/precision) + (1.0/recall) )
  
  if debug:
    print("Precision = TP/(TP+FP) = fraction of predicted true actually true ",precision)
    print("Recall = TP/(TP+FN) = fraction of true class predicted to be true ",recall)
    print("F1 score = ",f1_score)

  #
  # Get the ROC curve.  We will use the sklearn function to do this
  from sklearn import metrics
  #fpr_train, tpr_train, thresholds_train = metrics.roc_curve(y_train, y_train_score, pos_label=1)
  fpr, tpr, thresholds = metrics.roc_curve(y, y_score, pos_label=1)
  #
  # Get the auc
  auc = metrics.roc_auc_score(y, y_score)
  if debug:
    print("AUC this sample: ",auc)
  
  return precision,recall,auc,fpr, tpr, thresholds

#  The runFitter Method
We will use the same form of the runFitter method we used in our decision tree workbook.

In [6]:

def runFitter(estimator,X_train,y_train,X_test,y_test,debug=False):
#
# Now fit to our training set
  estimator.fit(X_train,y_train)
#
# Now predict the classes and get the score for our traing set
  y_train_pred = estimator.predict(X_train)
  y_train_score = estimator.predict_proba(X_train)[:,1]   # NOTE: some estimators have a predict_prob method instead od descision_function
#
# Now predict the classes and get the score for our test set
  y_test_pred = estimator.predict(X_test)
  y_test_score = estimator.predict_proba(X_test)[:,1]

#
# Now get the performaance
  precision_test,recall_test,auc_test,fpr_test, tpr_test, thresholds_test\
    = binaryPerformance(y_test,y_test_pred,y_test_score,debug)
  precision_train,recall_train,auc_train,fpr_train, tpr_train, thresholds_train\
    = binaryPerformance(y_train,y_train_pred,y_train_score,debug)
#
# Decide what you want to return: for now, just precision, recall, and auc for both test and train
  results = {
      'precision_train':precision_train,
      'recall_train':recall_train,
      'auc_train':auc_train,
      'fpr_train':fpr_train, 
      'tpr_train':tpr_train, 
      'thresholds_train':thresholds_train,
      'precision_test':precision_test,
      'recall_test':recall_test,
      'auc_test':auc_test,
      'fpr_test':fpr_test, 
      'tpr_test':tpr_test, 
      'thresholds_test':thresholds_test}

  return results
  

# Prepare the data
As usual we shuffle it first and then dump the dataframe into an **X** features numpy array and a **y** labels number array.

In [7]:
from sklearn.utils import shuffle
dfCombinedShuffle = shuffle(dfCombined,random_state=42)    # by setting the random state we will get reproducible results

X = dfCombinedShuffle.as_matrix(columns=dfCombinedShuffle.columns[:num_features])
y = dfCombinedShuffle['class'].values

/apps/project/ondemand/python/3.6-conda5.2/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.



# Training a single random forest, with k-fold validation.
Let's train a single random forest estimator, including cross validation.   But first we have to define the settings for the random forest estimator.   Here is the call definition from sklearn:

    class sklearn.ensemble.RandomForestClassifier(n_estimators=’warn’, criterion=’gini’, max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=’auto’, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, class_weight=None)
    
You will notice many parameters which are there to control the individual trees in the forest, such as **max_depth**,  **max_leaf_nodes** etc.   These serve exaclty the same purpose as described in the decision tree workbook.  We will only modify one of these - the **max_depth**  of the trees.   

The other primary parameter is **n_estimators** which is simply the number of trees in the forest.   We will typically set this to 100, but forests with 500, 1000, or even 5000 trees are not uncommon.   The bigger the forest the longer it will take to train.   This can be parallelized, which is what **n_jobs** is for.   For now, we will leave this and all of the other parameters at their default values, unless specified otherwise.

In [8]:
from sklearn.model_selection import StratifiedKFold
kfolds = 5
skf = StratifiedKFold(n_splits=kfolds)

In [9]:
from sklearn.ensemble import RandomForestClassifier
estimator = RandomForestClassifier(n_estimators=100, max_depth=6,random_state=42)

avg_precision_train = 0.0
avg_recall_train = 0.0
avg_auc_train = 0.0
avg_precision_test = 0.0
avg_recall_test = 0.0
avg_auc_test = 0.0
numSplits = 0.0
#
# Now loop
for train_index, test_index in skf.split(X, y):
  print("Training")
  numSplits += 1
  X_train = X[train_index]
  y_train = y[train_index]
  X_test = X[test_index]
  y_test = y[test_index]
  
#
# Now fit to our training set
  results = runFitter(estimator,X_train,y_train,X_test,y_test,debug=False)

  avg_precision_train += results['precision_train']
  avg_recall_train += results['recall_train']
  avg_auc_train += results['auc_train']
#
  avg_precision_test += results['precision_test']
  avg_recall_test += results['recall_test']
  avg_auc_test += results['auc_test']
#
avg_precision_train /= numSplits
avg_recall_train /= numSplits
avg_auc_train /= numSplits
avg_precision_test /= numSplits
avg_recall_test /= numSplits
avg_auc_test /= numSplits
# 
# Now print
print("Precision train/test ",round(avg_precision_train,3),round(avg_precision_test,3))
print("Recall train/test    ",round(avg_recall_train,3),round(avg_recall_test,3))
print("AUC train/test       ",round(avg_auc_train,3),round(avg_auc_test,3))

Training
Training
Training
Training
Training
Precision train/test  0.987 0.972
Recall train/test     0.926 0.913
AUC train/test        0.993 0.976


# Overfitting vs Underfitting
As with decision trees, we would like to make our parameter choices such that we are neither significantly over or under-fitting our training data.   

To keep things simple, we will again vary only one parameter - the max_depth of the trees in the forest.   To choose the optimal point, we will plot both training and testing error vs max_depth.   

We will again use 1-precision, 1-recall, and 1-AUC, as a measure of the **error** in our models.

In [10]:
#
# Get our estimator and predict
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
#
# Create a dataframe to store our results
dfError = pd.DataFrame(columns=['max_depth','trainError_pre','testError_pre',
                                    'trainError_rec','testError_rec',
                                    'trainError_auc','testError_auc'])

for max_depth in range(1,20):
  print("training with max depth =",max_depth)
  estimator = RandomForestClassifier(n_estimators=100,random_state=42,max_depth=max_depth)
  avg_precision_train = 0.0
  avg_recall_train = 0.0
  avg_auc_train = 0.0
  avg_precision_test = 0.0
  avg_recall_test = 0.0
  avg_auc_test = 0.0
  numSplits = 0.0
#
# Now loop
  for train_index, test_index in skf.split(X, y):
    numSplits += 1
    X_train = X[train_index]
    y_train = y[train_index]
    X_test = X[test_index]
    y_test = y[test_index]

#
# Now fit to our training set
    results = runFitter(estimator,X_train,y_train,X_test,y_test,debug=False)
    #print("results",results)

    avg_precision_train += results['precision_train']
    avg_recall_train += results['recall_train']
    avg_auc_train += results['auc_train']
#
    avg_precision_test += results['precision_test']
    avg_recall_test += results['recall_test']
    avg_auc_test += results['auc_test']
#
  avg_precision_train /= numSplits
  avg_recall_train /= numSplits
  avg_auc_train /= numSplits
  avg_precision_test /= numSplits
  avg_recall_test /= numSplits
  avg_auc_test /= numSplits
#
# Fill dataframe
  dfError = dfError.append({
     'max_depth':max_depth,
     'trainError_pre':1.0-avg_precision_train,'testError_pre':1.0-avg_precision_test,
     'trainError_rec':1.0-avg_recall_train,'testError_rec':1.0-avg_recall_test,
     'trainError_auc':1.0-avg_auc_train,'testError_auc':1.0-avg_auc_test
      }, ignore_index=True)
# 
# Now print
print(dfError.head)

training with max depth = 1
training with max depth = 2
training with max depth = 3
training with max depth = 4
training with max depth = 5
training with max depth = 6
training with max depth = 7
training with max depth = 8
training with max depth = 9
training with max depth = 10
training with max depth = 11
training with max depth = 12
training with max depth = 13
training with max depth = 14
training with max depth = 15
training with max depth = 16
training with max depth = 17
training with max depth = 18
training with max depth = 19
<bound method NDFrame.head of     max_depth  trainError_pre  testError_pre  trainError_rec  testError_rec  \
0         1.0        0.028581       0.028632        0.129347       0.131802   
1         2.0        0.018795       0.021595        0.116230       0.117767   
2         3.0        0.021729       0.025693        0.093961       0.099461   
3         4.0        0.023982       0.027947        0.081453       0.089699   
4         5.0        0.020762    

# Plotting results

Lets look at the error measures:
1.  train/test Error using precision: remember that **precision** = TP/(TP+FP) = "Total actual positive found as positive" / "Total our model identified as positive".   It is the fraction of identified postives that are truly positive.   **(1-precision)** is then the **error** - the fraction of our identified postives that are incorrect.
2.  train/test Error using recall: remember that **recall** = TP/(TP+FN) = "Total actual positive found as positive" / "Total actual positive".   It is the fraction of actual positives that our model managed to identify.   **(1-recall)** is then the **error** - the fraction of actual postives that we failed to identify.
3.  train/test Error using AUC: remember that AUC measures the probability that a randomly chosen positive example is properly ranked above a randomly chosen negative example.   **(1-AUC)** is then the probability that we will fail to do this.

In [11]:
import plotly.plotly as py
import numpy as np
from plotly.offline import iplot
import plotly.graph_objs as go

# Create a trace
trace1 = go.Scatter(
    x = dfError['max_depth'],
    y = dfError['trainError_pre'],
    mode = 'lines',
    name = "Training error"
)
# Create a trace
trace2 = go.Scatter(
    x = dfError['max_depth'],
    y = dfError['testError_pre'],
    mode = 'lines',
    name = "Testing Error"
)

layout = dict(
    title='Error (Precision) vs Model Complexity',
    xaxis=dict(title='max_depth'),
    yaxis=dict(title='Error (fraction)')
)

data = [trace1, trace2]
iplot(dict(data=data,layout=layout),validate=False)

# Create a trace
trace1 = go.Scatter(
    x = dfError['max_depth'],
    y = dfError['trainError_rec'],
    mode = 'lines',
    name = "Training error"
)
# Create a trace
trace2 = go.Scatter(
    x = dfError['max_depth'],
    y = dfError['testError_rec'],
    mode = 'lines',
    name = "Testing Error"
)

layout = dict(
    title='Error (Recall) vs Model Complexity',
    xaxis=dict(title='max_depth'),
    yaxis=dict(title='Error (fraction)')
)

data = [trace1, trace2]
iplot(dict(data=data,layout=layout),validate=False)
# Create a trace
trace1 = go.Scatter(
    x = dfError['max_depth'],
    y = dfError['trainError_auc'],
    mode = 'lines',
    name = "Training error"
)
# Create a trace
trace2 = go.Scatter(
    x = dfError['max_depth'],
    y = dfError['testError_auc'],
    mode = 'lines',
    name = "Testing Error"
)

layout = dict(
    title='Error (AUC) vs Model Complexity',
    xaxis=dict(title='max_depth'),
    yaxis=dict(title='Error (fraction)')
)

data = [trace1, trace2]
iplot(dict(data=data,layout=layout),validate=False)


# Feature Importance (1)
A very useful property of random forests is that they can be used to assess **feature importance**.   To get this from a trained random forest, we need to first train a single random forest, using our optimal settings for our hyperparameters.

Note that in the code below:
1.  We call runFitter once.
2.  We feed runFitter the same X,y arrays for both training and testing (since we don't really care about the testing set anymore).
3.  Remember that the expected performance is **not** the result printed out below, but the avereged result from k-fold validation from above (at our optimal hyperparameter settings).


In [12]:
from sklearn.utils import shuffle
dfCombinedShuffle = shuffle(dfCombined,random_state=42)    # by setting the random state we will get reproducible results

X = dfCombinedShuffle.as_matrix(columns=dfCombinedShuffle.columns[:num_features])
y = dfCombinedShuffle['class'].values

from sklearn.ensemble import RandomForestClassifier
estimator = RandomForestClassifier(n_estimators=100, max_depth=6,random_state=42,oob_score=True)

results = runFitter(estimator,X,y,X,y,debug=False)

print("Precision:",results['precision_train'])
print("Recall:   ",results['recall_train'])
print("AUC:      ",results['auc_train'])


/apps/project/ondemand/python/3.6-conda5.2/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.



Precision: 0.9837239583333334
Recall:    0.9219035997559487
AUC:       0.9918978409505044


# Feature Importance (2)
Now that we have trained our random forest estimator, we can get the list of feature importances by accessing the **attribute** of our estimator like this:
    * estimator.feature_importances_

This is a **list**, ordered exactly as our features that we fed to the estimator - so it has the same order as **dfCombinedShuffle.columns** (up to but not including the **class** column).

The code below shows how to access the feaure importance, as well as how to order it from maximum to minimum.

There is a very interesting discussion of issues around the calculation of feature importances here:  https://explained.ai/rf-importance/index.html

In [13]:
importanceByName = {}
print("unsorted importance")
for name,importance in zip(dfCombinedShuffle.columns[:num_features],estimator.feature_importances_):
  importanceByName[name] = importance
  print("Name,importance",name,round(importance,3))
#
# Now sort and print
print()
print("Sorted importance")
for name in sorted(importanceByName, key=importanceByName.get, reverse=True):
  print("Name,importance",name,round(importanceByName[name],3))


unsorted importance
Name,importance Profile_mean 0.165
Name,importance Profile_stdev 0.015
Name,importance Profile_skewness 0.459
Name,importance Profile_kurtosis 0.182
Name,importance DM_mean 0.068
Name,importance DM_stdev 0.066
Name,importance DM_skewness 0.019
Name,importance DM_kurtosis 0.022
Name,importance random 0.006

Sorted importance
Name,importance Profile_skewness 0.459
Name,importance Profile_kurtosis 0.182
Name,importance Profile_mean 0.165
Name,importance DM_mean 0.068
Name,importance DM_stdev 0.066
Name,importance DM_kurtosis 0.022
Name,importance DM_skewness 0.019
Name,importance Profile_stdev 0.015
Name,importance random 0.006


# Additional things to try:
Here are some things to explore to make sure you understand how things work:
1.   You can add a second random variable to the features.   Do both random features end up at the bottom of importances?
2.  You can remove the random variable and make sure your results are unaffected.
3.   You can try different settings of trees: ntrees=10, or ntrees=200.   How much do your results change?